In [21]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dense

from imutils import paths


import numpy as np
import argparse
import cv2
import os
import glob
import pymc3 as pm

from edward.models import Bernoulli, Normal
import edward as ed
import tensorflow as tf

In [3]:
def image_to_feature_vector(image, size=(32, 32)): return cv2.resize(image, size).flatten()

In [5]:
cats = map(image_to_feature_vector, [cv2.imread(cat) for cat in glob.glob('data/cats/*')])
dogs = map(image_to_feature_vector, [cv2.imread(dog) for dog in glob.glob('data/dogs/*')])

data = np.vstack((cats,dogs))
labels = ['cat']*len(cats) + ['dog']*len(dogs)

In [6]:
le = LabelEncoder()
labels = le.fit_transform(labels)
data = np.array(data) / 255.0
labels = np_utils.to_categorical(labels, 2)

In [7]:
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [8]:
model = Sequential()
model.add(Dense(768, input_dim=3072, init="uniform",
	activation="relu"))
model.add(Dense(384, init="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

/Users/kbrusch/Desktop/_notebook/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(768, activation="relu", kernel_initializer="uniform", input_dim=3072)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kbrusch/Desktop/_notebook/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(384, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.


In [9]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
model.fit(trainData, trainLabels, nb_epoch=50, batch_size=128, verbose=1)

[INFO] compiling model...


/Users/kbrusch/Desktop/_notebook/lib/python2.7/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
300/300 [==============================] - 0s - loss: 0.7087 - acc: 0.5300     
Epoch 2/50
300/300 [==============================] - 0s - loss: 0.6989 - acc: 0.5167     
Epoch 3/50
300/300 [==============================] - 0s - loss: 0.6636 - acc: 0.5933     
Epoch 4/50
300/300 [==============================] - 0s - loss: 0.6651 - acc: 0.6233     
Epoch 5/50
300/300 [==============================] - 0s - loss: 0.6483 - acc: 0.6333     
Epoch 6/50
300/300 [==============================] - 0s - loss: 0.6798 - acc: 0.5567     
Epoch 7/50
300/300 [==============================] - 0s - loss: 0.6930 - acc: 0.6067     
Epoch 8/50
300/300 [==============================] - 0s - loss: 0.6275 - acc: 0.6900     
Epoch 9/50
300/300 [==============================] - 0s - loss: 0.6320 - acc: 0.6667     
Epoch 10/50
300/300 [==============================] - 0s - loss: 0.6167 - acc: 0.7033     
Epoch 11/50
300/300 [==============================] - 0s - loss: 0.6099 - acc: 0.7067   

In [60]:
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy*100))


In [40]:
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainData.shape

(300, 3072)

In [61]:
#model = Sequential()
#model.add(Dense(768, input_dim=3072, init="uniform",
#	activation="relu"))
#model.add(Dense(384, init="uniform", activation="relu"))
#model.add(Dense(2))
#model.add(Activation("softmax"))

from edward.models import Bernoulli, Normal
import edward as ed
import tensorflow as tf


#N = 3072  # number of data points
#D = 300   # number of features

#z = Normal(loc=tf.zeros([N, D]), scale=tf.ones([N, D]))
#h = Dense(768, input_dim=3072, activation='relu')(z)

#x = Bernoulli(logits=Dense(2)(h))


b = Sequential()
#b.add(Dense(768, input_dim=3072,activation="relu"))
#b.add(Dense(384, activation="relu"))
#b.add(Dense(2))
#b.add(Activation("softmax"))








In [87]:
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

def neural_network(X):
  h = tf.tanh(tf.matmul(X, W_0) + b_0)
  h = tf.tanh(tf.matmul(h, W_1) + b_1)
  h = tf.matmul(h, W_2) + b_2
  return tf.reshape(h, [-1])

N = 400  # number of data points, 400 if straight from 'data'
D = 3072   # number of features


# MODEL
W_0 = Normal(loc=tf.zeros([D, 10]), scale=tf.ones([D, 10]))
W_1 = Normal(loc=tf.zeros([10, 10]), scale=tf.ones([10, 10]))
W_2 = Normal(loc=tf.zeros([10, 1]), scale=tf.ones([10, 1]))
b_0 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_1 = Normal(loc=tf.zeros(10), scale=tf.ones(10))
b_2 = Normal(loc=tf.zeros(1), scale=tf.ones(1))

X = tf.placeholder(tf.float32, [N, D])
y = Bernoulli(logits=neural_network(X))

# INFERENCE
qW_0 = Normal(loc=tf.Variable(tf.random_normal([D, 10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, 10]))))
qW_1 = Normal(loc=tf.Variable(tf.random_normal([10, 10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10, 10]))))
qW_2 = Normal(loc=tf.Variable(tf.random_normal([10, 1])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10, 1]))))
qb_0 = Normal(loc=tf.Variable(tf.random_normal([10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_1 = Normal(loc=tf.Variable(tf.random_normal([10])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_2 = Normal(loc=tf.Variable(tf.random_normal([1])),
              scale=tf.nn.softmax(tf.Variable(tf.random_normal([1]))))



#type(data)
labels = [1]*len(cats) + [0]*len(dogs)
np.array(labels).shape
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                     W_1: qW_1, b_1: qb_1,
                     W_2: qW_2, b_2: qb_2}, data={X: data, y: labels})
#inference = ed.MonteCarlo({W_0: qW_0, b_0: qb_0,
#                     W_1: qW_1, b_1: qb_1,
#                     W_2: qW_2, b_2: qb_2}, data={X: data, y: labels})


#inference.run()



In [27]:
tf.zeros([100, 10])

<tf.Tensor 'zeros_3:0' shape=(100, 10) dtype=float32>

In [11]:
X, Y = data, labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5)
ann_input = theano.shared(data)
ann_output = theano.shared(labels)

# Initialize random weights between each layer
init_1 = np.random.randn(X.shape[1], 768)
init_2 = np.random.randn(768, 384)
init_out = np.random.randn(384)

NameError: name 'theano' is not defined

In [10]:
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    weights_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                             shape=(X.shape[1], 768), 
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    weights_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                            shape=(768, 384), 
                            testval=init_2)
    
    # Weights from hidden lay2er to output
    weights_2_out = pm.Normal('w_2_out', 0, sd=1, 
                              shape=(384,), 
                              testval=init_out)
    
    # Build neural-network using tanh activation function
    act_1 = T.nnet.relu(T.dot(ann_input, 
                         weights_in_1))
    act_2 = T.nnet.relu(T.dot(act_1, 
                         weights_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, 
                                   weights_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', 
                       act_out,

                       observed=ann_output)
"""
model = Sequential()
model.add(Dense(768, input_dim=3072, init="uniform",
	activation="relu"))
model.add(Dense(384, init="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))
"""    

/usr/local/lib/python2.7/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


ValueError: Input dimension mis-match. (input[0].shape[1] = 2, input[1].shape[1] = 400)